# Timeseries extrapolation

I made this notebook after working on [Advent of Code 2023, Day 9](https://adventofcode.com/2023/day/9).

The puzzle is about finding differences of differences of differences, but then also extrapolating first left, then right.

[Here's](https://github.com/kwinkunks/aoc23/blob/main/day09.py) how I solved the puzzle at the time. But recursion is much prettier.

We'll also solve the extrapolation problem with statistical methods instead.

First, here's the test data and the expected results.

In [1]:
import numpy as np

test_data = """0 3 6 9 12 15
1 3 6 10 15 21
10 13 16 21 30 45"""

y = [18, 28, 68]

X = np.array(list(map(int, test_data.split()))).reshape(3, -1)
X

array([[ 0,  3,  6,  9, 12, 15],
       [ 1,  3,  6, 10, 15, 21],
       [10, 13, 16, 21, 30, 45]])

---

## Naive approach

We can compute the chain of differences, a bit like Pascal's triangle but with subtraction instead of addition. This was my solution to the Advent of Code puzzle. 

In [10]:
from typing import Callable

def predict(history: list[int],
            prequel: bool=False,
            f: Callable=sum
    ) -> int:
    "f is cumulating function"
    X = [history[prequel-1]]
    while True:
        history = [j - i for i, j in zip(history, history[1:])]
        if not any(history): break
        X.append(history[prequel-1])
    return f(X)

list(map(predict, X))

[np.int64(18), np.int64(28), np.int64(68)]

### Recursive version

Compute the differences, as in the puzzle description, but do it recursively.

In [11]:
def extrapolate(l):
    diffs = [b - a for a, b in zip(l, l[1:])]
    return l[-1] + extrapolate(diffs) if len(l) else 0

list(map(extrapolate, X))

[np.int64(18), np.int64(28), np.int64(68)]

---

## Pure autoregression

I don't have good intuition for why this is so approximate.

In [14]:
import statsmodels.tsa.api as sm

for xi in X:
    model = sm.AutoReg(xi, lags=1).fit()
    yhat = model.predict(xi.size, xi.size)
    print(yhat)

[18.]
[28.88888889]
[69.34146341]


---

## ARIMA

- **AR** - autoregression
- **I** - integrated (differences)
- **MA** - moving average

Read about it [on Wikipedia](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average). It works nicely here...

In [17]:
extra = []
for xi in X:
    model = sm.ARIMA(xi, order=(0, 6, 0)).fit()
    yhat = model.predict(xi.size, xi.size)
    extra.append(yhat)

print(extra)

[array([18.]), array([28.]), array([68.])]


### ARIMA backwards (i.e. to the left, back in time)

In [ ]:
extra = []
for xi in X:
    model = sm.ARIMA(xi[::-1], order=(0, 6, 0)).fit()
    yhat = model.predict(xi.size, xi.size)
    extra.append(yhat)

print(extra)
sum(extra)

[array([-3.]), array([-3.24131915e-11]), array([5.])]


array([2.])

---

## Finite difference

Turns out finite differences can be computed using only coefficients and the dot product. Amazing and basically magic.

In [ ]:
w = [-1, 6, -15, 20, -15, 6]

X @ w

array([18, 28, 68])

Those weights must be computed, for example with [Bengt Fornberg's algorithm](https://gist.github.com/kwinkunks/6401dd08444da97331ad0188501be957). See Fornberg, B (1988). Generation of Finite Difference Formulas on Arbitrarily Spaced Grids, Mathematics of Compuation 51 (184), p 699-706, doi: 10.1090/S0025-5718-1988-0935077-0. Here's my implementation:

In [19]:
def weights(z, x, m=0):
    """
    Fornberg finite difference weights.
    F90: https://github.com/bjodah/finitediff/blob/master/src/finitediff_fort.f90

    Arguments:
        z: location where approximations are to be accurate
        x: grid point locations, found in x(0:n)
        m: highest derivative for which weights are sought

    Returns:
        array

    References:
        B Fornberg (1988). Generation of Finite Difference Formulas on
            Arbitrarily Spaced Grids, Mathematics of Computation 51 (184),
            p 699-706, doi: 10.1090/S0025-5718-1988-0935077-0.

    Example:
        >>> weights(6, range(6), 0)
        array([[ -1],
               [  6],
               [-15],
               [ 20],
               [-15],
               [  6]])
    """
    n = len(x) - 1
    c = np.zeros((n+1, m+1))
    c1 = 1
    c4 = x[0] - z
    c[0, 0] = 1

    for i in range(1, n+1):
        mn = min(i, m)
        c2 = 1
        c5 = c4
        c4 = x[i] - z
        for j in range(i):
            c3 = x[i] - x[j]
            c2 *= c3
            if j == i - 1:
                for k in range(mn, 0, -1):
                    c[i, k] = c1 * (k * c[i-1, k-1] - c5 * c[i-1, k]) / c2
                c[i, 0] = -c1 * c5 * c[i-1, 0] / c2
            for k in range(mn, 0, -1):
                c[j, k] = (c4 * c[j, k] - k * c[j, k-1]) / c3
            c[j, 0] = c4 * c[j, 0] / c3
        c1 = c2

    return c.astype(int)

In [20]:
weights(6, range(6), 0)

array([[ -1],
       [  6],
       [-15],
       [ 20],
       [-15],
       [  6]])

---

&copy; 2024 Matt Hall, licensed CC BY, please re-use this work.